In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","8g") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .getOrCreate()

[ WARN] Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
[ WARN] Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
url='neo4j://localhost:7692'
password = 'password'
checkpoint1 = "/home/worker2/Desktop/thesis/K18/File_K18/sample_crawl_dataset/checkpoint/checkpoint1"
checkpoint2 = "/home/worker2/Desktop/thesis/K18/File_K18/sample_crawl_dataset/checkpoint/checkpoint2"
checkpoint3 = "/home/worker2/Desktop/thesis/K18/File_K18/sample_crawl_dataset/checkpoint/checkpoint3"

In [3]:
schema = "id INT,username STRING,email STRING,career STRING,skills STRING"

In [4]:
df = spark.readStream.option('header','true') \
    .format('csv') \
    .schema(schema) \
    .option("maxFilesPerTrigger", 1) \
    .option('inferSchema', 'True') \
    .option("escape", '"')\
    .load('/home/worker2/Desktop/thesis/K18/File_K18/sample_crawl_dataset/user/')

In [5]:
#query = user_info.writeStream.format("console").start()
#query.awaitTermination()

In [6]:
from pyspark.sql.functions import explode, split
user_info = df.select(df.id,df.username, df.email)

In [7]:
from pyspark.sql.functions import *
query1 = user_info.writeStream.format("org.neo4j.spark.DataSource")\
    .option("save.mode", "Overwrite") \
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("checkpointLocation", checkpoint1) \
    .option("node.keys", "id")\
    .option("labels",":User")\
    .start()
#query1.awaitTermination()

[ WARN] spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [8]:
user_favor = df.select(df.id, df.career)

In [9]:
query2 =user_favor.writeStream.format("org.neo4j.spark.DataSource")\
    .option("save.mode", "Overwrite") \
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("checkpointLocation", checkpoint2) \
    .option("relationship", 'FAVOR_IN')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':User')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Career')\
    .option('relationship.target.node.keys','career:name')\
    .option('relationship.target.save.mode','Match')\
    .start()

[ WARN] spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [10]:
user_skills= df.select(df.id, split(df.skills,",").alias("skills"))

In [11]:
user_skills = user_skills.select(user_skills.id, explode(user_skills.skills).alias("skills"))

In [12]:
query3 = user_skills.writeStream.format("org.neo4j.spark.DataSource")\
    .option("save.mode", "Overwrite") \
    .option("url", url)\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", password)\
    .option("checkpointLocation", checkpoint3) \
    .option("relationship", 'HAVE_SKILL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':User')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Competency')\
    .option('relationship.target.node.keys','skills:name')\
    .option('relationship.target.save.mode','Match')\
    .start()

[ WARN] spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [13]:
query1.awaitTermination()
query2.awaitTermination()
query3.awaitTermination()

#spark.streams.awaitAnyTermination()
